# OpenAI Agent with Query Engine Tools

## Build Query Engine Tools

In [1]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

from llama_index.tools import QueryEngineTool, ToolMetadata

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/lyft')
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir='./storage/uber')
    uber_index = load_index_from_storage(storage_context)
    
    index_loaded = True
except:
    index_loaded = False

In [3]:
if not index_loaded:
    # load data 
    lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
    uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
    
    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)
    
    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(
            name='lyft_10k', 
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(
            name='uber_10k', 
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
]


## Setup OpenAI Agent

In [6]:
from llama_index.agent import OpenAIAgent

In [7]:
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

## Let's Try It Out!

In [ ]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Compare and contrast Lyft and Uber's revenue growth in 2021, then give an analysis


=== Calling Function ===
Calling function: lyft_10k with args: {
  "input": "What was Lyft's revenue growth in 2021?"
}
Got output: 
Lyft's revenue growth in 2021 was 36%.
=== Calling Function ===
Calling function: uber_10k with args: 
{
  "input": "What was Uber's revenue growth in 2021?"
}
Got output: 
Uber's revenue growth in 2021 was 57%.
Assistant: In 2021, both Lyft and Uber experienced significant revenue growth. Lyft's revenue grew by 36%, while Uber's revenue grew by 57%. 

The higher revenue growth rate of Uber indicates that it had a stronger performance in terms of generating revenue compared to Lyft. This could be attributed to several factors, including Uber's larger market presence and global reach. Uber operates in more countries and cities compared to Lyft, which allows it to capture a larger customer base and generate more revenue.

However, it's important to note that revenue growth alone does not provide a complete picture of a company's financial performance. Other